一、依赖安装

In [1]:
!pip install tokenizers tqdm

二、分块读取工具函数

In [2]:
import os
from typing import BinaryIO

def find_chunk_boundaries(
    file: BinaryIO,
    desired_num_chunks: int,
    split_special_token: bytes,
) -> list[int]:
    """
    将文件切分为可以独立计数的块（Chunk）。
    如果边界发生重叠（例如文件太小或分隔符太稀疏），返回的块数量可能会少于预期。
    """
    # 确保传入的分隔符是字节串类型，因为文件是以二进制模式读取的
    assert isinstance(split_special_token, bytes), "必须使用字节串（bytes）表示特殊 Token"

    # 获取文件的总字节大小
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0) # 回到文件开头

    # 计算初步的理想块大小（字节数）
    chunk_size = file_size // desired_num_chunks

    # 初始的边界猜测：根据块大小进行均匀分布
    # 边界数组包含起始位置 0 和结束位置 file_size
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size # 确保最后一个边界精确指向文件末尾

    mini_chunk_size = 4096  # 每次向后搜索的缓冲区大小4k字节

    # 遍历除了开头和结尾之外的所有中间边界点
    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # 跳转到初步猜测的边界位置
        
        while True:
            # 读取一小块数据进行扫描
            mini_chunk = file.read(mini_chunk_size)

            # 如果读到了文件末尾（EOF），说明后面没有分隔符了，直接设为文件末尾
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # 在当前小块数据中查找指定的分隔符
            found_at = mini_chunk.find(split_special_token)
            
            if found_at != -1:
                # 如果找到了分隔符，将边界调整到该分隔符的确切位置
                chunk_boundaries[bi] = initial_position + found_at
                break
            
            # 如果没找到，继续向后移动指针进行下一轮搜索
            initial_position += mini_chunk_size

    # set()：去除重复的边界，防止多个猜测点指向同一个分隔符
    # sorted()：确保边界按从小到大的顺序排列
    return sorted(set(chunk_boundaries))

三、按块读取文本

In [3]:
import time
from tqdm import tqdm

def iter_text_chunks_with_monitor(
    file_path: str,
    chunk_size: int = 1_000_000,  # 1MB
    log_every: int = 5,          # 每N个chunk打印一次
):
    start_time = time.time()
    bytes_processed = 0
    chunk_count = 0

    with open(file_path, "r", encoding="utf-8") as f:
        buffer = []
        buffer_size = 0

        for line in f:
            buffer.append(line)
            buffer_size += len(line)
            bytes_processed += len(line)

            if buffer_size >= chunk_size:
                yield "".join(buffer)
                buffer = []
                buffer_size = 0
                chunk_count += 1

                if chunk_count % log_every == 0:
                    log_status(
                        prefix="📘 分词器流式处理",
                        bytes_processed=bytes_processed,
                        start_time=start_time,
                    )

        if buffer:
            yield "".join(buffer)


四、训练BPE分词器

Step1:在不修改tokenizer内部实现的前提下，实时监控内存占用与数据吞吐量，理解tokenizer训练的真实系统行为。

In [4]:
# 安装监控需要的依赖
!pip install psutil

In [5]:
# 当前进程所占内存
import psutil
import os

def get_memory_mb():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

In [6]:
# 日志状态函数输出（内存占用、处理数据量、处理速度）
import time
def log_status(prefix, bytes_processed, start_time):
    elapsed = time.time() - start_time  
    mb = bytes_processed / 1024 / 1024
    throughput = mb / elapsed if elapsed > 0 else 0.0 # 计算吞吐量即每秒处理的数据量
    mem = get_memory_mb() 

    print(
        f"{prefix} | "
        f"mem={mem:7.1f} MB | "
        f"data={mb:8.1f} MB | "
        f"speed={throughput:6.2f} MB/s"
    )

Step2:训练BPE Tokenizer

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC
#from tqdm import tqdm

In [8]:
def train_bpe_tokenizer(
    train_file: str,
    val_file: str | None = None,
    vocab_size: int = 50257,
    num_chunks: int = 8,
    output_dir: str = "./bpe_tokenizer",
):
    os.makedirs(output_dir, exist_ok=True)

    special_tokens = [
        "<|endoftext|>",
        "<|unk|>",
        "<|pad|>",
        "<|bos|>",
        "<|eos|>",
    ]

    tokenizer = Tokenizer(BPE(unk_token="<|unk|>"))
    tokenizer.normalizer = NFKC()

    # 设计GPT-2风格的BPE Tokenizer
    tokenizer.pre_tokenizer = ByteLevel(add_prefix_space=True)
    tokenizer.decoder = ByteLevelDecoder()

    trainer = BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=special_tokens,
        show_progress=True, 
    )

    def text_iterator():
        # 训练集
        for chunk in iter_text_chunks_with_monitor(
            train_file,
            chunk_size=1_000_000,    # 每次处理大小约为1MB的原始数据，处理完就会清除数据方便继续处理
            log_every=20,
        ):
            yield chunk

        # 验证集（可选）
        if val_file is not None:
            for chunk in iter_text_chunks_with_monitor(
                val_file,
                chunk_size=1_000_000,
                log_every=10,
            ):
                yield chunk


    print("🚀 开始训练BPE Tokenizer...")
    tokenizer.train_from_iterator(text_iterator(), trainer=trainer)
    print("✅ BPE Tokenizer训练完成")

    tokenizer.save(os.path.join(output_dir, "tokenizer.json"))
    print(f"💾 分词器已保存至{output_dir}/tokenizer.json")

    return tokenizer
if __name__ == "__main__":
    train_path = "./TinyStoriesV2-GPt4-train.txt"
    val_path = "./TinyStoriesV2-GPt4-valid.txt"
    tokenizer = train_bpe_tokenizer(
        train_file=train_path,
        val_file=val_path,
        vocab_size=50257,     # 词表大小（通常设为32000或50257等）
        num_chunks=16,        # 读取文件时的分块数量（建议根据内存大小调整）
        output_dir="./bpe_tokenizer",
    )

🚀 开始训练BPE Tokenizer...
📘 分词器流式处理 | mem=   89.2 MB | data=    19.1 MB | speed=217.19 MB/s
📘 分词器流式处理 | mem=  108.4 MB | data=    38.2 MB | speed=226.17 MB/s
📘 分词器流式处理 | mem=  126.9 MB | data=    57.2 MB | speed=228.38 MB/s
📘 分词器流式处理 | mem=  146.1 MB | data=    76.3 MB | speed=228.28 MB/s
📘 分词器流式处理 | mem=  165.3 MB | data=    95.4 MB | speed=229.61 MB/s
📘 分词器流式处理 | mem=  184.3 MB | data=   114.5 MB | speed=230.22 MB/s
📘 分词器流式处理 | mem=  204.4 MB | data=   133.5 MB | speed=218.34 MB/s
📘 分词器流式处理 | mem=  223.6 MB | data=   152.6 MB | speed=223.29 MB/s
📘 分词器流式处理 | mem=  242.7 MB | data=   171.7 MB | speed=218.71 MB/s
📘 分词器流式处理 | mem=  261.9 MB | data=   190.8 MB | speed=208.73 MB/s
📘 分词器流式处理 | mem=  281.0 MB | data=   209.8 MB | speed=212.70 MB/s
📘 分词器流式处理 | mem=  300.1 MB | data=   228.9 MB | speed=215.92 MB/s
📘 分词器流式处理 | mem= 1382.3 MB | data=   248.0 MB | speed= 18.86 MB/s
📘 分词器流式处理 | mem= 1105.1 MB | data=   267.1 MB | speed= 20.00 MB/s
📘 分词器流式处理 | mem=  146.9 MB | data=   286.1 MB | speed

五、验证训练的BPE Tokenizer

In [45]:
encoded = tokenizer.encode(" Hello, world! <|endoftext|>")
print(encoded.tokens)    # 打印编码后的token序列
print(encoded.ids)       # 打印编码后的ID序列
# print(tokenizer.decode([1501])) # 输出应该是" world"（前面带个空格）

# 将Ġ替换回空格
clean_tokens = [t.replace('Ġ', ' ') for t in encoded.tokens]
print(clean_tokens)

['ĠHello', ',', 'Ġworld', '!', 'Ġ', '<|endoftext|>']
[8431, 16, 1501, 5, 149, 0]
[' Hello', ',', ' world', '!', ' ', '<|endoftext|>']


Q: 为什么在GTP-2正则化风格中训练出来的tokenizer会在单词前添加一个'Ġ'？
A: 因为对于模型而言，在同一个单词中，没有带空格和带有空格表示的Token是不同的，因此为了区分这两种情况就会添加了一个'Ġ'来表示带空格的Token（比如"hello"和"Ġhello"，前者表示"hello"这个单词，后者表示" hello"这个单词）。

In [12]:
# Token统计函数
def analyze_tokenizer(tokenizer, texts):
    lengths = [len(tokenizer.encode(t).ids) for t in texts]
    return {
        "avg_tokens": sum(lengths) / len(lengths), # 平均处理token数
        "max_tokens": max(lengths),                # 最大token数，用于设置最大处理序列长度（决定是否截断序列处理）
    }

In [15]:
import random

def load_stories(file_path, num_samples=None):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    stories = [s.strip() for s in content.split("<|endoftext|>") if s.strip()]
    
    if num_samples is not None:
        n = min(num_samples, len(stories))
        stories = random.sample(stories, k=n)
    return stories

train_path = "./TinyStoriesV2-GPT4-train.txt"
val_path = "./TinyStoriesV2-GPT4-valid.txt"
test_texts = load_stories(val_path, num_samples=10)
train_texts = load_stories(train_path, num_samples=20)
print(f"随机抽取的训练样本数: {len(train_texts)}")
print(f"随机抽取的验证样本数: {len(test_texts)}")

# 分析训练集和验证集的token统计
train_stats = analyze_tokenizer(tokenizer, train_texts)
val_stats = analyze_tokenizer(tokenizer, test_texts)
print("训练集统计:", train_stats)
print("验证集统计:", val_stats)

随机抽取的训练样本数: 20
随机抽取的验证样本数: 10
训练集统计: {'avg_tokens': 197.0, 'max_tokens': 308}
验证集统计: {'avg_tokens': 208.2, 'max_tokens': 434}
